In [1]:
import pandas as pd
from ptrail.core.TrajectoryDF import PTRAILDataFrame

# The following link contains the traffic dataset
url_traffic = 'https://raw.githubusercontent.com/YakshHaranwala/PTRAIL/main/examples/data/traffic.csv'
data_traffic = pd.read_csv(url_traffic)

traffic_df = PTRAILDataFrame(data_set=data_traffic,
                        latitude='lat',
                        longitude='lon',
                        datetime='datetime',
                        traj_id='id')
print(traffic_df)

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [ ]:
from ptrail.features.kinematic_features import KinematicFeatures as spatial

traffic_df = spatial.create_acceleration_column(traffic_df)
traffic_df = spatial.create_bearing_column(traffic_df)
traffic_df = spatial.create_jerk_column(traffic_df)
trajectories = traffic_df.index.unique(level="traj_id")

# Creating a dataset with mean of values
traj_df_list = []
for traj in trajectories:
    traj_df = pd.DataFrame(traffic_df.loc[[traj]].mean()).transpose()
    traj_df['traj_id'] = traj
    traj_df_list.append(traj_df)

mean_df = pd.concat(traj_df_list, ignore_index=True)
drop_list = ['vehicle_type', 'lon', 'lat', 'kilopost',
             'detected_flag', 'traj_id']
mean_df = mean_df.drop(drop_list, axis=1)

In [ ]:
# Importing pycaret and creating a model to
# estimate the length of the vehicle
from pycaret.regression import *

length_reg = setup(data = mean_df,
                   target = 'vehicle_length',
                   normalize=True)

In [ ]:
# Lets try to find the best model
compare_models()

In [ ]:
# Huber was the best one during the run so
# we'll create a new and tune it
huber = create_model('huber')

In [ ]:
tuned_huber = tune_model(huber)

In [ ]:
# We can perform some plots to undestand it better
plot_model(tuned_huber)

In [ ]:
plot_model(tuned_huber, plot = 'error')

In [ ]:
plot_model(tuned_huber, plot='feature')